- Ref: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen3_(4B)-GRPO.ipynb#scrollTo=DkIvEkIIkEyB

Instal unsloth

In [1]:
import torch
print(torch.__version__)
# uv add "unsloth[cu124-torch251] @ git+https://github.com/unslothai/unsloth.git"

2.6.0+cu124


Pre-fine-tune the model to make GRPO follow the desired output format — this speeds up GRPO.

In [2]:
import unsloth
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/sean/MLAlgoFromScratch/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 08-26 09:03:06 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 08-26 09:03:06 [__init__.py:239] Automatically detected platform cuda.


2025-08-26 09:03:08,403	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset, load_dataset

from trl import SFTTrainer, SFTConfig

In [4]:
max_seq_length = 2048

In [5]:
# key: vllm==0.8.5.post1
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='unsloth/Qwen3-4B-base',
    max_seq_length=512,
    load_in_4bit=True,
    fast_inference=True, # Enable vLLM fast inference
    max_lora_rank=8,
    gpu_memory_utilization=0.6 # Reduce if OOM
)

Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.7.8: Fast Qwen3 patching. Transformers: 4.54.0. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.581 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen3-4b-base-unsloth-bnb-4bit with actual GPU utilization = 59.0%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.58 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 5.94 GB. Also swap space = 0 GB.
WARNING 08-26 09:03:23 [config.py:2972] Casting torch.bfloat16 to torch.float16.
INFO 08-26 09:03:34 [config.py:71

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:13<00:00, 13.26s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:13<00:00, 13.26s/it]

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.44s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.44s/it]


INFO 08-26 09:03:56 [punica_selector.py:18] Using PunicaWrapperGPU.


INFO 08-26 09:03:56 [model_runner.py:1140] Model loading took 3.1446 GiB and 18.560822 seconds
INFO 08-26 09:04:00 [worker.py:287] Memory profiling takes 3.77 seconds
INFO 08-26 09:04:00 [worker.py:287] the current vLLM instance can use total_gpu_memory (14.58GiB) x gpu_memory_utilization (0.59) = 8.60GiB
INFO 08-26 09:04:00 [worker.py:287] model weights take 3.14GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 1.05GiB; the rest of the memory reserved for KV Cache is 4.39GiB.
INFO 08-26 09:04:01 [executor_base.py:112] # cuda blocks: 1995, # CPU blocks: 0
INFO 08-26 09:04:01 [executor_base.py:117] Maximum concurrency for 512 tokens per request: 62.34x
INFO 08-26 09:04:01 [vllm_utils.py:669] Unsloth: Running patched vLLM v0 `capture_model`.
INFO 08-26 09:04:01 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eage

Capturing CUDA graph shapes: 100%|██████████| 27/27 [00:13<00:00,  1.96it/s]

INFO 08-26 09:04:14 [model_runner.py:1592] Graph capturing finished in 14 secs, took 4.24 GiB
INFO 08-26 09:04:14 [vllm_utils.py:676] Unsloth: Patched vLLM v0 graph capture finished in 14 secs.


INFO 08-26 09:04:15 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 19.00 seconds
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'post_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'post_feedforward_layernorm']


In [6]:
max_dim = tokenizer.model_max_length
print(f"Tokenizer can handle up to {max_dim} tokens.")

Tokenizer can handle up to 32768 tokens.


In [7]:
tokenizer.chat_template is None
# ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! 

True

In [8]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560, padding_idx=151654)
    (layers): ModuleList(
      (0-1): 2 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2560, out_features=4096, bias=False)
          (k_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=2560, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (up_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (down_proj): Linear(in_features=9728, out_features=2560, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RM

In [9]:
lora_rank = 8  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

Unsloth 2025.7.8 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [10]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 2560, padding_idx=151654)
        (layers): ModuleList(
          (0-1): 2 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2560, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
  

In [11]:
reasoning_start = "<start_working_out>" # Acts as <think>
reasoning_end   = "<end_working_out>"   # Acts as </think>
solution_start  = "<SOLUTION>"
solution_end    = "</SOLUTION>"

system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your solution between {solution_start}{solution_end}"""
system_prompt

'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>'

In [12]:
"""
- Always includes a system prompt (either from messages or a default one).
- Marks assistant turns with an eos_token so the model knows where each reply ends.
- Optionally adds a “start reasoning” marker to signal when the model should generate output.
- Uses placeholders that get replaced at runtime so you can change system prompts or special tokens without editing the Jinja template.
"""

chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

# Replace without specific template:
chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{reasoning_start}'")
tokenizer.chat_template = chat_template

In [13]:
# Turn a structured list of messages into the exact string prompt the model will see.
tokenizer.apply_chat_template([
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"{reasoning_start}I think it's 2.{reasoning_end}{solution_start}2{solution_end}"},
    {"role" : "user", "content" : "What is 2+2?"},
], tokenize = False, add_generation_prompt = True)

"You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION><|endoftext|>What is 1+1?<start_working_out>I think it's 2.<end_working_out><SOLUTION>2</SOLUTION><|endoftext|>What is 2+2?<start_working_out>"

In [66]:
# load dataset
dataset = load_dataset("unsloth/OpenMathReasoning-mini", split = "cot")
dataset.to_pandas()

,expected_answer,problem_type,problem_source,generation_model,pass_rate_72b_tir,problem,generated_solution,inference_mode
0,14,has_answer_extracted,aops_c4_high_school_math,DeepSeek-R1,0.96875,Given $\sqrt{x^2+165}-\sqrt{x^2-52}=7$ and $x$...,"<think>\nOkay, let's see. I need to solve the ...",cot
1,convergent,has_answer_extracted,aops_c7_college_math,DeepSeek-R1,0.96875,Let \( \sum a_n \) be a convergent series with...,"<think>\nOkay, so I have this problem here whe...",cot
2,\(\frac{\pi \ln a}{2a}\),has_answer_extracted,aops_c7_college_math,DeepSeek-R1,0.96875,"For \( a > 0 \), find the value of \( \int_0^{...","<think>\nOkay, so I need to solve the integral...",cot
3,\(+\infty\),has_answer_extracted,aops_c7_college_math,DeepSeek-R1,0.96875,Calculate $\lim_{n\to\infty}\sqrt[n]{n!}$.,"<think>\nOkay, so I need to find the limit as ...",cot
4,\( n = 1 \),has_answer_extracted,aops_c6_high_school_olympiads,DeepSeek-R1,0.96875,Find all positive integers \( n \) such that \...,"<think>\nOkay, so I need to find all positive ...",cot
...,...,...,...,...,...,...,...,...
19247,4,has_answer_extracted,aops_c4_high_school_math,DeepSeek-R1,0.96875,A bus left point X for point Y. Two hours late...,"<think>\nOkay, let's tackle this problem step ...",cot
19248,18,has_answer_extracted,aops_c4_high_school_math,DeepSeek-R1,0.96875,Each interior angle of a regular n-gon measure...,"<think>\nOkay, let's see. I need to find the n...",cot
19249,\( e^3 - 1 \),has_answer_extracted,aops_c7_college_math,DeepSeek-R1,0.96875,Evaluate the series \( \sum_{k=1}^{\infty}\fra...,"<think>\nOkay, so I need to evaluate the serie...",cot
19250,0.8960,has_answer_extracted,aops_c4_high_school_math,DeepSeek-R1,0.96875,Find the probability that the second blue resu...,"<think>\nOkay, so I need to find the probabili...",cot


In [73]:
dataset = dataset[:2]
dataset = Dataset.from_dict(dataset)

Note
- I think the reason of using math here is the answer is precise
- If the result is text, we may compute the text similarity to confirm the answer ?!

In [74]:
dataset = dataset.to_pandas()[
    ["expected_answer", "problem", "generated_solution"]
]

# Try converting to number - if not, replace with NaN
is_number = pd.to_numeric(pd.Series(dataset["expected_answer"]), errors = "coerce").notnull()
# Select only numbers
dataset = dataset.iloc[np.where(is_number)[0]]

In [75]:
def format_dataset(x):
    expected_answer = x["expected_answer"]
    problem = x["problem"]

    # Remove generated <think> and </think>
    thoughts = x["generated_solution"]
    thoughts = thoughts.replace("<think>", "").replace("</think>", "")

    # Strip newlines on left and right
    thoughts = thoughts.strip()
    # Add our custom formatting
    final_prompt = \
        reasoning_start + thoughts + reasoning_end + \
        solution_start + expected_answer + solution_end
    return [
        {"role" : "system",    "content" : system_prompt},
        {"role" : "user",      "content" : problem},
        {"role" : "assistant", "content" : final_prompt},
    ]

dataset["Messages"] = dataset.apply(format_dataset, axis = 1)

In [76]:
test_msg = dataset.iloc[0].Messages
test_msg

[{'role': 'system',
  'content': 'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>'},
 {'role': 'user',
  'content': 'Given $\\sqrt{x^2+165}-\\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.'},
 {'role': 'assistant',
  'content': "<start_working_out>Okay, let's see. I need to solve the equation √(x² + 165) - √(x² - 52) = 7, and find all positive values of x. Hmm, radicals can be tricky, but maybe if I can eliminate the square roots by squaring both sides. Let me try that.\n\nFirst, let me write down the equation again to make sure I have it right:\n\n√(x² + 165) - √(x² - 52) = 7.\n\nOkay, so the idea is to isolate one of the radicals and then square both sides. Let me try moving the second radical to the other side:\n\n√(x² + 165) = 7 + √(x² - 52).\n\nNow, if I square both sides, maybe I can get rid of the square ro

Check to see if it worked

In [77]:
# default of tokenize is True
tokenizer.apply_chat_template(test_msg, tokenize=False)

"You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION><|endoftext|>Given $\\sqrt{x^2+165}-\\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.<start_working_out>Okay, let's see. I need to solve the equation √(x² + 165) - √(x² - 52) = 7, and find all positive values of x. Hmm, radicals can be tricky, but maybe if I can eliminate the square roots by squaring both sides. Let me try that.\n\nFirst, let me write down the equation again to make sure I have it right:\n\n√(x² + 165) - √(x² - 52) = 7.\n\nOkay, so the idea is to isolate one of the radicals and then square both sides. Let me try moving the second radical to the other side:\n\n√(x² + 165) = 7 + √(x² - 52).\n\nNow, if I square both sides, maybe I can get rid of the square roots. Let's do that:\n\n(√(x² + 165))² = (7 + √(x² - 52))².\n\nSimplifying the left side:\n\nx² + 

In [78]:
tokenized_result = tokenizer.apply_chat_template(test_msg)
print(f"Tokenized result: {tokenized_result[:10]}... \nwith dimension: {len(tokenized_result)}")

Tokenized result: [2610, 525, 2661, 264, 3491, 624, 38687, 911, 279, 3491]... 
with dimension: 1682


Limit sequence length to max_seq_length / 2 during pre-fine-tuning to avoid overly long reasoning.

In [79]:
dataset = dataset.loc[[len(tokenizer.apply_chat_template(x)) < max_seq_length for x in dataset['Messages']]]
dataset.shape

(2, 4)

Tokenize data and convert it to a HF compatible dataset format

In [80]:
dataset['text'] = tokenizer.apply_chat_template(dataset['Messages'].values.tolist(), tokenize=False)

In [81]:
dataset.head()

,expected_answer,problem,generated_solution,Messages,text
0,14,Given $\sqrt{x^2+165}-\sqrt{x^2-52}=7$ and $x$...,"<think>\nOkay, let's see. I need to solve the ...","[{'role': 'system', 'content': 'You are given ...",You are given a problem.\nThink about the prob...
1,-2,Find the value of the parameter $a$ for which ...,"<think>\nOkay, so I need to find the value of ...","[{'role': 'system', 'content': 'You are given ...",You are given a problem.\nThink about the prob...


In [82]:
dataset = Dataset.from_pandas(dataset)

In [83]:
dataset

Dataset({
    features: ['expected_answer', 'problem', 'generated_solution', 'Messages', 'text', '__index_level_0__'],
    num_rows: 2
})

Pre fine-tune the model so it follows the custom GRPO formatting

In [ ]:
# check the result before pre fine-tune
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)
text = "What is the sqrt of 101?"
output = model.fast_generate(
    [text], 
    sampling_params=sampling_params, 
    lora_request=None
)
print(output[0].outputs[0].text)


Processed prompts: 100%|██████████| 1/1 [00:09<00:00,  9.68s/it, est. speed input: 1.03 toks/s, output: 28.41 toks/s]

 - Quora
What is the sqrt of 101?
6 Answers
Best
Gaurav Bakliwal
Answered 2 years ago · Author has 526 answers and 384.9K answer author has 20.8K answers and 10M answer views 2 y
What is the sqrt of 101?
First of all 101 is not a square. It is pretty close to 100 which is 10×10. However 101 is 4 more than 100, so take √(100+4)
=(√100)+√4
=10 + 2
100.50.00.25.00…
√101 =10.05
Related
What is the square root of 614? (in square root symbol)
What is the square root of 614? (in square root symbol)
2,08.8
614 = 4 *153.5
2,√153.5
614 is a prime number.
Related Question
What is the square root of 614? (in squared symbol)
2,20


In [84]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args = SFTConfig(
        dataset_text_field='text',
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8, # update after accumulation step
        warmup_steps=5,
        num_train_epochs=2,
        learning_rate=2e-4,
        logging_steps=5,
        optim='adamw_8bit',
        weight_decay=0.01,
        lr_scheduler_type='linear',
        seed=3407,
        report_to='none'
    )

)

Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 2/2 [00:01<00:00,  1.25 examples/s]


In [85]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2 | Num Epochs = 2 | Total steps = 2
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 16,515,072 of 4,038,983,168 (0.41% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss


TrainOutput(global_step=2, training_loss=0.864276647567749, metrics={'train_runtime': 4.9164, 'train_samples_per_second': 0.814, 'train_steps_per_second': 0.407, 'total_flos': 25306746322944.0, 'train_loss': 0.864276647567749})

In [86]:
text = tokenizer.apply_chat_template(
    dataset[0]["Messages"][:2],
    tokenize=False,
    add_generation_prompt=True # add this then the ending is "<start_working_out>"
)
text

'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION><|endoftext|>Given $\\sqrt{x^2+165}-\\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.<start_working_out>'

In [ ]:
# from transformers import TextStreamer
# _ = model.generate(
#     **tokenizer(text, return_tensors = "pt").to("cuda"),
#     temperature = 0,
#     max_new_tokens = 1024,
#     streamer = TextStreamer(tokenizer, skip_prompt = False),
# )

In [87]:
# clean memory
del dataset
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [88]:
dataset = load_dataset("open-r1/DAPO-Math-17k-Processed", "en", split = "train")
dataset

Dataset({
    features: ['prompt', 'solution', 'data_source', 'source_prompt', 'ability', 'reward_model', 'extra_info'],
    num_rows: 14116
})

In [89]:
dataset = dataset[:2]

In [90]:
dataset = Dataset.from_dict(dataset)

In [91]:
# the overall prompt (including problem and system prompt) in the OpenR1 datatset
dataset[0]["source_prompt"]

[{'content': 'Solve the following math problem step by step. The last line of your response should be of the form Answer: $Answer (without quotes) where $Answer is the answer to the problem.\n\nIn triangle $ABC$, $\\sin \\angle A = \\frac{4}{5}$ and $\\angle A < 90^\\circ$. Let $D$ be a point outside triangle $ABC$ such that $\\angle BAD = \\angle DAC$ and $\\angle BDC = 90^\\circ$. Suppose that $AD = 1$ and that $\\frac{BD}{CD} = \\frac{3}{2}$. If $AB + AC$ can be expressed in the form $\\frac{a\\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$.\n\nRemember to put your answer on its own line after "Answer:".',
  'role': 'user'}]

In [92]:
# the problem
print(dataset[0]["prompt"])

In triangle $ABC$, $\sin \angle A = \frac{4}{5}$ and $\angle A < 90^\circ$. Let $D$ be a point outside triangle $ABC$ such that $\angle BAD = \angle DAC$ and $\angle BDC = 90^\circ$. Suppose that $AD = 1$ and that $\frac{BD}{CD} = \frac{3}{2}$. If $AB + AC$ can be expressed in the form $\frac{a\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$.


Synthesis the dataset with customized system prompt and quetion(the name is prompt in the dataset)

In [93]:
dataset = dataset.map(lambda x: {
    "prompt": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": x["prompt"]}
    ],
    "answer": x['solution']
})

Map: 100%|██████████| 2/2 [00:00<00:00, 283.65 examples/s]


Extract the final answer from a structured text that contains both:
- a reasoning section (e.g., <REASONING>...</REASONING>)
- and a solution section (e.g., <SOLUTION>...</SOLUTION>)

In [94]:
import re

# Add optional EOS token matching
solution_end_regex = r"</SOLUTION>[\s]{0,}" + \
    "(?:" + re.escape(tokenizer.eos_token) + ")?"

match_format = re.compile(
    rf"{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end_regex}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)
match_format

re.compile(r'<end_working_out>.*?<SOLUTION>(.+?)</SOLUTION>[\s]{0,}(?:<\|endoftext\|>)?[\s]{0,}$',
           re.MULTILINE|re.DOTALL|re.UNICODE)

In [95]:
# test case
match_format.findall(
    "Let me think!<end_working_out>"\
    f"<SOLUTION>\n2\n</SOLUTION>",
)

['\n2\n']

In [96]:
match_format.findall(
    "Let me think!<end_working_out>"\
    f"<SOLUTION>364</SOLUTION>",
)

['364']

Create reward function to match the format 
- +3 points if the output matches the full format exactly
- Partial reward if key format tags (<REASONING>, </REASONING>, etc.) are present (each adds +0.5).
- Penalty (-1.0) if a required tag is missing or appears more than once.

In [97]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

In [98]:
def match_format_partially(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!

        # No need to reward <start_working_out> since we always prepend it!
        # score += 0.5 if response.count(reasoning_start) == 1 else -1.0
        score += 0.5 if response.count(reasoning_end)   == 1 else -1.0
        score += 0.5 if response.count(solution_start)  == 1 else -1.0
        score += 0.5 if response.count(solution_end)    == 1 else -1.0
        scores.append(score)
    return scores

Reward of penalize the generated content

In [112]:
def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]['content']
    responses = [completion[0]['content'] for completion in completions]

    extracted_responses = [
        result.group(1)
        if (result := match_format.search(r)) is not None else None for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(-2.0) # Note that it is append to scores not score, which means no answer=-2, then next run
            continue
        
        # 5 points for correct answer
        if guess  == true_answer:
            score += 5.0

        # Match if spaces are seen but less reward
        elif guess.strip() == true_answer.strip():
            score += 3.5
        
        else:
            # Reward the answer if it is close via ratios
            try:
                ratio = float(guess) / float(true_answer)
                if ratio >= 0.9 and ratio <= 1.1:
                    score += 2.0
                elif ratio >= 0.8 and ratio <= 1.2:
                    score += 1.5
                else:
                    score += -2.5
            except:
                score += -4.5
        scores.append(score)

    return scores

Remove commas such as 123,456 or extract the number in a sentence for example "The solution is $20" -> 20

In [113]:
# Match numbers like 17, 123,456, 0.34, -0.234, 1,234,567.89
match_numbers = re.compile(r"-?(?:\d{1,3}(?:,\d{3})+|\d+)(?:\.\d+)?", re.MULTILINE | re.DOTALL)

def find_numbers_clean(text):
    """Return numbers as strings with commas removed."""
    return [n.replace(",", "") for n in match_numbers.findall(text)]

# If you'd like floats instead, use this:
def find_numbers_float(text):
    return [float(n.replace(",", "")) for n in match_numbers.findall(text)]

# Tests
print(find_numbers_clean("<SOLUTION>  0.34  </SOLUTION>"))      # ['0.34']
print(find_numbers_clean("<SOLUTION>  123,456  </SOLUTION>"))   # ['123456']
print(find_numbers_clean("<SOLUTION>  -0.234  </SOLUTION>"))    # ['-0.234']
print(find_numbers_clean("<SOLUTION> The solution is 17  </SOLUTION>"))  # ['17']

['0.34']
['123456']
['-0.234']
['17']


In [114]:
global PRINTED_TIMES
PRINTED_TIMES = 0 # a counter that increments every time check_numbers is called.
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5 # ets the frequency (here 5).  

In [ ]:
def check_number(prompts, completions, answer, **kwargs):
    """
    Evaluate numeric answers extracted from model completions against ground truth.

    Scoring (per item):
      - 3.5  if the extracted numeric guess equals the true answer (after stripping & removing commas)
      - -1.5 if a number was extracted but it does not equal the true answer
      - -2.5 if no number could be extracted
      - 0    if parsing fails unexpectedly

    Args:
        prompts:
            A sequence of message lists. This function reads the question text from
            `prompts[0][-1]["content"]`
            Example shape: `[[{"role":"user","content":"..."}], ...]`
        completions:
            A sequence of generation lists. Each item should be a list whose first element
            has a `"content"` string, accessed as `completion[0]["content"]`.
            Example shape: `[[{"role":"assistant","content":"..."}], ...]`
        answers:
            Sequence of ground-truth answers (strings or numbers). Must align 1:1 with `completions`.

    Returns:
        List[float]: Scores aligned with `completions` / `answers`.

    Notes:
        - Compares numeric equality after normalizing commas in guesses.
        - `answers` are cast to float via `float(str(answer).strip())`.
    """
    question = prompts[0][-1]['content']
    responses = [completion[0]["content"] for completion in completions] # how to confirm it is right?


    # apply reg.search to extract the answer, since the completion may contain other string.
    extracted_responses = [
        guess.group(1) if (guess := match_numbers.search(r)) is not None else None for r in responses
    ]
    
    scores = []
    # Print only every few steps
    global PRINTED_TIMES
    global PRINT_EVERY_STEPS
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        print(
            '*'*20 + f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}"
        )
    PRINTED_TIMES += 1

    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(-2.5)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            # Remove commas like in 123,456
            guess       = float(guess.strip().replace(",", ""))
            scores.append(3.5 if guess == true_answer else -1.5)
        except:
            scores.append(0)
            continue
    return scores

In [116]:
global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5

def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    # Print only every few steps
    global PRINTED_TIMES
    global PRINT_EVERY_STEPS
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        print(
            '*'*20 + f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}"
        )
    PRINTED_TIMES += 1

    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(-2.5)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            # Remove commas like in 123,456
            guess       = float(guess.strip().replace(",", ""))
            scores.append(3.5 if guess == true_answer else -1.5)
        except:
            scores.append(0)
            continue
    return scores

Warlus operation explanation

In [117]:
import re

# Regex to capture a number (with optional decimal part)
match_numbers = re.compile(r"(-?\d+(?:\.\d+)?)")

responses = [
    "The answer is 42.",
    "I think it's -3.14",
    "No number here!",
    "Maybe 1,234 is correct"
]
extracted_responses = [
    guess.group(1)
    if (guess := match_numbers.search(r)) is not None else None
    for r in responses
]

"""
Warlus operator (`:=`):
does two things at once:
1. Runs match_numbers.search(r)
2. Saves the result into guess
3. Returns the result so it can be used immediately in the expression
"""

print(extracted_responses)


['42', '-3.14', None, '1']


In [118]:
numbers = [1, 2, 3, 4]
result = []
for n in numbers:
    x = n * n
    if x > 5:
        result.append(x)
print(result)

[9, 16]


In [119]:
numbers = [1, 2, 3, 4]

result = [x for n in numbers if (x := n*n) > 5]

print(result)  

[9, 16]


For stable traninig, we want to restrict the model_length
1. remove the top 10% long prompts
2. check the max model input length # it didn't involved in this tutorial

In [120]:
# batch_size = 1000 (default)
tokenized = dataset.map(
    lambda x: {"tokens": tokenizer.apply_chat_template(x['prompt'], add_generation_prompt=True, tokenized=True)}, # add_generation_prompt = True for gettin the max possible length
    batched= True
)

# calculate the length
tokenized = tokenized.map(lambda x: {"L": len(x['tokens'])})

maximum_length = int(np.quantile(tokenized['L'], 0.9))
print("Max Length = ", maximum_length)

# Filter only samples smaller than 90% max length
dataset = dataset.select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

Map: 100%|██████████| 1/1 [00:00<00:00, 181.41 examples/s]

Max Length =  127


### Train the model

In [121]:
# We use GRPOTrainer in here, from-scratch implementation will be another folder

max_prompt_length = maximum_length + 1 # + 1 just in case!
max_completion_length = max_seq_length - max_prompt_length

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

In [122]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    vllm_sampling_params=vllm_sampling_params,
    temperature=1.0,
    learning_rate=5e-6,
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type='linear',
    optim='adamw_8bit',
    logging_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1, # Increase to 4 for smoother training
    num_generations=4,
    max_prompt_length=max_prompt_length,
    max_completion_length=max_completion_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 100,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [123]:
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_partially,
        check_answer,
        check_number
    ],
    args=training_args,
    train_dataset=dataset,
    
    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1 | Num Epochs = 100 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 16,515,072 of 4,038,983,168 (0.41% trained)


NameError: name 'prompt' is not defined

In [ ]:
for item in dataset:
    len_prompt = len(item['prompt'])
    len_sol = len(item['solution'])

    if len_prompt == 0:
        print(0)

    if len_sol == 0:
        print(1)

In [ ]:
len(dataset[0]['prompt'])

2